# Задание 1 (4 балла)

1) Прочтите датасет titanic.csv

2) Разделите на train-val выборки с помощью train_test_split функции, процент test = 0.2

3) Закодируйте категориальные признаки, стандартизуйте действительные признаки, заполните пропущенные данные, удалите лишение признаки. 

4) Обучите алгоритм логистической регрессии на тренировочной выборке. Какие признаки больше всего влияют на качество предсказаний алгоритма?

5) Рассчитайте precision алгоритма

In [2069]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as pltcolors
import category_encoders as ce
from sklearn.metrics import precision_score
from scipy import optimize
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [2070]:
# !wget https://raw.githubusercontent.com/rolandmueller/titanic/main/titanic3.csv

In [2071]:
# чтение данных
df = pd.read_csv('titanic3.csv')
print(df.columns.shape)
df.head()

(14,)


,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [2072]:
# отбор признаков
features = ['survived', 'sex', 'age', 'sibsp', 'parch', 'fare']
df[features]

,survived,sex,age,sibsp,parch,fare
0,1,female,29.0000,0,0,211.3375
1,1,male,0.9167,1,2,151.5500
2,0,female,2.0000,1,2,151.5500
3,0,male,30.0000,1,2,151.5500
4,0,female,25.0000,1,2,151.5500
...,...,...,...,...,...,...
1304,0,female,14.5000,1,0,14.4542
1305,0,female,NaN,1,0,14.4542
1306,0,male,26.5000,0,0,7.2250
1307,0,male,27.0000,0,0,7.2250


In [2073]:
# разделение на train и test
X_train, X_test, y_train, y_test = train_test_split(df[features], df['pclass'], test_size=0.2, random_state=0)
X_train.shape, X_test.shape

((1047, 6), (262, 6))

In [2074]:
# One Hot Encodig для пола пассажиров

# ПОЧЕМУ ЭТО НЕ РАБОТАЕТ?!
#one_hot_encoder = OneHotEncoder(sparse_output=False)
#X_train['sex'] = one_hot_encoder.fit_transform(X_train)
#X_train


X_train.loc[X_train['sex'] == 'male', 'sex'] = int(1)
X_train.loc[X_train['sex'] == 'female', 'sex'] = int(0)

X_test.loc[X_test['sex'] == 'male', 'sex'] = int(1)
X_test.loc[X_test['sex'] == 'female', 'sex'] = int(0)

In [2075]:
result_train = pd.concat([X_train.isnull().sum(), X_train.isnull().mean()], axis=1)
result_train = result_train.rename(index=str,columns={0:'total missing',1:'proportion'})
print(result_train.apply(lambda x: x[x > 0]))
# в трейне у 207 людей нет возраста, заменим его средним
# отсутсвующее значение fare удалим

      total missing  proportion
age             206    0.196753
fare              1    0.000955


In [2076]:
result_test = pd.concat([X_test.isnull().sum(), X_test.isnull().mean()], axis=1)
result_test = result_test.rename(index=str,columns={0:'total missing',1:'proportion'})
print(result_test.apply(lambda x: x[x > 0]))

# в тесте у 56 людей нет возраста, заменим его средним

     total missing  proportion
age             57    0.217557


In [2077]:
# удаление 1-го значения fare
null_index_train = X_train[X_train['fare'].isnull()].index.to_numpy()
X_train = X_train.dropna(subset=['fare'], axis=0, inplace=False)
y_train = y_train.drop(null_index_train)

In [2078]:
# замена возраста средним
X_train['age'] = X_train['age'].fillna(value=X_train['age'].mean())
X_test['age'] = X_test['age'].fillna(value=X_test['age'].mean())


### Рассмотрим выбросы

In [2079]:
def outlier_detect_IQR(data, col, threshold=3):
    f = False
    # Рассчитываем межквартильный размах (IQR).
    IQR = data[col].quantile(0.75) - data[col].quantile(0.25)

    # Вычисляем верхнюю и нижнюю границы для определения выбросов.
    Lower_fence = data[col].quantile(0.25) - (IQR * threshold)
    Upper_fence = data[col].quantile(0.75) + (IQR * threshold)

    # Формируем кортеж с использованными границами.
    para = (Upper_fence, Lower_fence)

    # Создаем временный DataFrame, в котором для каждой строки указываем, превышает ли значение в столбце заданные границы.
    tmp = pd.concat([data[col] > Upper_fence, data[col] < Lower_fence], axis=1)

    # Находим индексы строк, содержащих хотя бы один выброс.
    outlier_index = tmp.any(axis=1)

    # Выводим информацию о найденных выбросах.
    if len(outlier_index.value_counts()) > 1:
        f = True
        #print(col, end= ": ")
        #print('Найдено выбросов:', outlier_index.value_counts().iloc[1])
    # Возвращаем серию с булевыми значениями и кортеж с границами.
    return outlier_index,f

In [2080]:
for i in features:
    index, find = outlier_detect_IQR(data=X_train, col=i, threshold=3)    
    if find:
        print(i, end= ": ")
        print('Найдено выбросов:', index.value_counts().iloc[1])

age: Найдено выбросов: 1
sibsp: Найдено выбросов: 15
parch: Найдено выбросов: 247
fare: Найдено выбросов: 71


Тут, я готов считать выбросами случай с parch, потому что есть случаи, когда у ребенка 9 родителей

In [2081]:
# Если у человека больше 2 родителей, то заменяем их количество на 2
X_train.loc[X_train['parch'] > 2, 'parch'] = 2


In [2082]:
for i in features:
    index, find = outlier_detect_IQR(data=X_test, col=i, threshold=3)    
    if find:
        print(i, end= ": ")
        print('Найдено выбросов:', index.value_counts().iloc[1])

age: Найдено выбросов: 1
parch: Найдено выбросов: 60
fare: Найдено выбросов: 13


In [2083]:
# Если у человека больше 2 родителей, то заменяем их количество на 2
X_test.loc[X_test['parch'] > 2, 'parch'] = 2

###

In [2084]:
# Стандартизация
for feature in features:
    st = StandardScaler().fit(X_train[[feature]])
    X_train[feature] = st.transform(X_train[[feature]])

for feature in features:
    st = StandardScaler().fit(X_test[[feature]])
    X_test[feature] = st.transform(X_test[[feature]])


In [2085]:
# Обучение

sk_model = SGDClassifier(loss='hinge', penalty='l2', fit_intercept=True, random_state=0, max_iter=500, learning_rate='constant', eta0=0.005)
result = sk_model.fit(X_train, y_train)
sk_model_acc = round(sk_model.score(X_test, y_test), 4)
print('Точность: ', sk_model_acc * 100)

Точность:  75.19


In [2115]:
weights = result.coef_
weights


array([[ 2.22064325e-01,  1.88591131e-01,  2.14373304e-01,
        -9.42125045e-01, -6.29515614e-01,  3.75782592e+00],
       [ 1.03767896e-02,  1.15168975e-02, -1.80110440e-02,
        -4.21118207e-03,  7.63753170e-03, -1.52136234e-02],
       [-5.68166955e-02, -2.06353241e-02, -7.88846860e-02,
         8.35092890e-01,  3.75962968e-01, -5.78382648e+00]])

насколько я понимаю, то первая строка показывает как признаки влияют на первый класс
вторая строка, на второй класс
третья строка, на трейти класс

In [2117]:
for raw in weights:
    for (i, x) in enumerate(raw):
       if (x > 0):
           print(features[i], end = " ")
    print()

survived sex age fare 
survived sex parch 
sibsp parch 


# Задание №2

Реализуйте функцию подбора оптимального порога с помощью FPR/TPR оценок на чистом Python. На вход функция принимает метки истинных классов и оценки алгоритма, на выходе функция должна определить лучший порог классификации. Подберите порог на тестовой выборке и заново оцените precision модели.

хоть оно и не работае, но за страние пару баллов пж 👉🏻👈🏻

In [2087]:
from sklearn import metrics

In [2088]:
def find_optimal_threshold(y_true, y_score):
    fpr, tpr, thresholds = metrics.roc_curve(y_true, y_score, pos_label=2)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]
    return optimal_threshold


# Пример использования

clf = SGDClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
optimal_threshold = find_optimal_threshold(y_test, y_pred)
y_pred_new = (clf.decision_function(X_test) >= optimal_threshold).astype(int)
y_pred_new_1 = []
for i in y_pred_new:
    if i[0] == 1:
        y_pred_new_1.append(1)
    elif i[1] == 1:
        y_pred_new_1.append(2)
    elif i[2] == 1:
        y_pred_new_1.append(3)
    
print(len(y_pred_new_1))
print(y_test.shape)


#precision_new = metrics.precision_score(y_test, y_pred_new)

67
(262,)
